In [1]:
# Install Hugging Face datasets and TensorFlow
!pip install datasets transformers tensorflow streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split


In [4]:
from google.colab import files
uploaded = files.upload()  # Upload the ZIP file (spoc.zip)

import zipfile
import os

with zipfile.ZipFile('spoc.zip', 'r') as zip_ref:
    zip_ref.extractall('spoc_dataset')

# Check extracted files
os.listdir('spoc_dataset/train')


Saving spoc.zip to spoc.zip


['split', 'spoc-train.tsv']

In [11]:
# Verify file existence
data_path = 'spoc_dataset/train/spoc-train.tsv'
print("Checking if the dataset file exists:", os.path.exists(data_path))


Checking if the dataset file exists: True


In [12]:
# Load the dataset
try:
    print("Attempting to read the dataset...")
    df = pd.read_csv(data_path, sep='\t')
    print("Dataset loaded successfully!")
    print(df.head())  # Display first few rows
    print("Dataset Columns:", df.columns)
except Exception as e:
    print("Error loading dataset:", e)
    raise

# Ensure required columns exist
if 'text' in df.columns and 'code' in df.columns:
    df.dropna(subset=['text', 'code'], inplace=True)
    df = df[['code', 'text']]  # Keep only relevant columns
else:
    raise ValueError("Dataset does not contain expected columns: 'text' and 'code'")

Attempting to read the dataset...
Dataset loaded successfully!
                                              text  \
0                in the function gcd(a,b=integers)   
1  if b=1 return a, else call function gcd(b, a%b)   
2                                              NaN   
3                                              NaN   
4               n , nn, ans = integers with ans =0   

                             code  workerid probid     subid  line  indent  
0         int gcd(int a, int b) {        38    13A  41120785     0       0  
1  return !b ? a : gcd(b, a % b);        38    13A  41120785     1       1  
2                               }        38    13A  41120785     2       0  
3                    int main() {        38    13A  41120785     3       0  
4             int n, nn, ans = 0;        38    13A  41120785     4       1  
Dataset Columns: Index(['text', 'code', 'workerid', 'probid', 'subid', 'line', 'indent'], dtype='object')


In [13]:
# Extracting relevant columns
pseudocode = df['text'].astype(str).tolist()
cpp_code = df['code'].astype(str).tolist()

# Splitting the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(pseudocode, cpp_code, test_size=0.1)


In [14]:
# Tokenizer
MODEL_NAME = "t5-small"  # You can use 't5-base' for better performance
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
# Tokenization function
def tokenize_function(texts, labels):
    inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    labels = tokenizer(labels, padding="max_length", truncation=True, max_length=256, return_tensors="pt")["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation
    return inputs, labels

In [16]:
# Custom Dataset Class
class PseudoCodeDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs, labels = tokenize_function(self.texts[idx], self.labels[idx])
        return {"input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
                "labels": labels.squeeze()}

In [17]:
# Creating DataLoaders
train_dataset = PseudoCodeDataset(train_texts, train_labels)
val_dataset = PseudoCodeDataset(val_texts, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [18]:
# Load the Transformer Model
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [19]:
# Training Setup
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_dataloader, val_dataloader, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader)}")

In [ ]:
# Train the model
train_model(model, train_dataloader, val_dataloader, epochs=3)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:

# Save the trained model
model.save_pretrained("trained_transformer")
tokenizer.save_pretrained("trained_transformer")

In [ ]:
# Function to generate C++ code from pseudocode
def generate_cpp_code(pseudocode):
    model.eval()
    inputs = tokenizer(pseudocode, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=256)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Example Usage
example_pseudocode = "create an integer n and read n"
generated_code = generate_cpp_code(example_pseudocode)
print("Generated C++ Code:\n", generated_code)